In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
trio_info=pd.read_excel('/storage/haplotypes/excels/trio_info.xlsx')
trio_info['relatives'] = trio_info['relatives'].replace(np.nan, 'None', regex=False)
trio_info['relatives']=trio_info['relatives'].astype(str)
trio_info.head()

,status,sample,relatives
0,Patient,22334455,None
1,Relative,77092483,77092484:child
2,Patient,77092484,77092482:father; 77092483:mother
3,Relative,77092482,77092484:child
4,Relative,77092481,77092479:child


In [73]:
patients=trio_info.loc[trio_info['status']=='Patient', 'sample'].to_list()

In [3]:
trios=defaultdict(dict)
for idx, row in trio_info.iterrows():
    if row['relatives']:
        relatives=[x for x in row['relatives'].split('; ')]
        for i in relatives:
            person=i.split(':')
            if len(person)>1:
                trios[row['sample']][person[1]] = int(person[0])
trios

defaultdict(dict,
            {77092483: {'child': 77092484},
             77092484: {'father': 77092482, 'mother': 77092483},
             77092482: {'child': 77092484},
             77092481: {'child': 77092479},
             77092480: {'child': 77092479},
             77092479: {'father': 77092480, 'mother': 77092481},
             77092478: {'child': 77092477},
             77092477: {'mother': 77092478},
             77092476: {'sibling': 77092477},
             77092473: {'child': 77092472, 'spouse': 77092474},
             77092472: {'mother': 77092473, 'father': 77092474},
             77092474: {'child': 77092472, 'spouse': 77092473},
             77092471: {'spouse': 77092470, 'child': 77092469},
             77092469: {'mother': 77092471, 'father': 77092470},
             77092470: {'spouse': 77092471, 'child': 77092469},
             77092468: {'child': 77092467, 'spouse': 77092475},
             77092475: {'spouse': 77092468, 'child': 77092467},
             77092467: {'fa

In [4]:
files_list= pd.read_excel('/storage/haplotypes/resolve_conf.xlsx', index_col=0)
calling_region = [32037643, 32041345]

In [5]:
def read_vcf(link, mode=None): 
    vcf = pd.read_csv(link, sep='\t', comment='#', header=None,
          names=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE'], index_col=False) 
    vcf = vcf.assign(GT=vcf['SAMPLE'].map(lambda x: x.split(':')[0]))
    if mode == 'hc':
        vcf = vcf.assign(AD=vcf['SAMPLE'].map(lambda x: x.split(':')[1]))
        vcf['DP'] = vcf['AD'].map(lambda x: sum(map(int, x.split(','))) if pd.notna(x) else None)
        vcf['DP'] = vcf['DP'].astype(int)
        vcf['GT'] = vcf['GT'].str.replace('/', '|', regex=False)
    if mode == 'dv':
        vcf = vcf.assign(AD=vcf['SAMPLE'].map(lambda x: x.split(':')[3]))
        vcf['DP'] = vcf['AD'].map(lambda x: sum(map(int, x.split(','))) if pd.notna(x) else None)
        vcf['GT'] = vcf['GT'].str.replace('|', '/', regex=False)
        vcf['DP'] = vcf['DP'].astype(int)
    if mode == 'bam':
        vcf = vcf.assign(AD=vcf['SAMPLE'].map(lambda x: x.split(':')[5]))
        vcf['DP'] = vcf['AD'].map(lambda x: sum(map(int, x.split(','))) if pd.notna(x) else None)
        vcf['DP'] = vcf['DP'].astype(int)
    
    return vcf

In [8]:
def gt_vectors(vcf, mode = None):
    gt_vector={}
    for idx, row in vcf.iterrows(): 
        if ',' in row['ALT']: 
            dp=int(row['AD'].split(',')[0])+int(row['AD'].split(',')[1]) + int(row['AD'].split(',')[2])
            if row['POS'] == 32038855:
                position = '32038855' + '_' + 'T' + "_" + 'TTTG'
                if mode == 'hc':
                    gt = '1|1'
                else:
                    gt='1/1'
                vaf=(int(row['AD'].split(',')[1]) + int(row['AD'].split(',')[2]))/dp
                gt_vector[position] = [gt, vaf, dp]
            else:
                for num, alt in enumerate(row['ALT'].split(',')):
                    position = str(row['POS']) + '_' + row['REF'] + "_" + alt
                    if mode == 'hc':
                        gt = '1|0'
                    else:
                        gt='1/0'
                    vaf=int(row['AD'].split(',')[num+1])/dp
                    gt_vector[position] = [gt, vaf, row['DP']]
                    
        else: 
            position=str(row['POS']) + '_' + row['REF'] + "_" + row['ALT']
            gt=row['GT']
            vaf=int(row['AD'].split(',')[1])/row['DP']
            gt_vector[position] = [gt, vaf, row['DP']]
    return gt_vector

In [7]:
#do I need DP checking here???
def check_relative_variant(position, hc_gen_info, dv_gen_info):
    print(f'checking relative variant with {hc_gen_info} AND {dv_gen_info}')
    hc_genotype=hc_gen_info[0]
    hc=hc_genotype.split('|')
    hc=int(hc[0])+int(hc[1])

    dv_genotype=dv_gen_info[0]
    dv=dv_genotype.split('/')
    dv=int(dv[0])+int(dv[1])

    if hc==dv:
        return hc_genotype
    else:
        if hc == 2:
            if hc_gen_info[1]>=0.85:
                if hc_gen_info[2] >= 30:
                    print(f'hc right with {hc_genotype}, VAF - {hc_gen_info[1]}, DP - {hc_gen_info[2]}')
                    return hc_genotype
                else:
                    print(f'hc call for {hc_genotype} with right VAF - {hc_gen_info[1]}, but low DP - {hc_gen_info[2]}')
                    return None
            if dv ==1 and (dv_gen_info[1] >= 0.2 and dv_gen_info[1] < 0.85):
                if dv_gen_info[2] >= 30:
                    print(f'dv right with {dv_genotype}, VAF - {dv_gen_info[1]}, DP - {dv_gen_info[2]}')
                    return dv_genotype
                else:
                    print(f'dv call for {dv_genotype} with right VAF - {dv_gen_info[1]}, but low DP - {dv_gen_info[2]}')
                    return None
            else:
                print(f'both callers have low qual variant in relative; HC: {hc_gen_info}, DV: {dv_gen_info}')
                return None
        if hc == 1:
            if (hc_gen_info[1] >= 0.2 and hc_gen_info[1] < 0.85):
                if hc_gen_info[2] >= 30:
                    print(f'hc_right with {hc_genotype}, VAF - {hc_gen_info[1]}, DP - {hc_gen_info[2]}')
                    return hc_genotype
                else:
                    print(f'hc call for {hc_genotype} with right VAF - {hc_gen_info[1]}, but low DP - {hc_gen_info[2]}')
                    return None
            if dv == 2 and dv_gen_info[1] >= 0.85:
                if dv_gen_info[2] >= 30:
                    print(f'dv right with {dv_genotype}, VAF - {dv_gen_info[1]}, DP - {dv_gen_info[2]}')
                    return dv_genotype
                else:
                    print(f'dv call for {dv_genotype} with right VAF - {dv_gen_info[1]}, but low DP - {dv_gen_info[2]}')
                    return None
            else:
                print(f'both callers have low qual variant in relative; HC: {hc_gen_info}, DV: {dv_gen_info}')
                return None

In [9]:
def check_inheritance(patient, pos, trios, gt_hc, gt_dv, gt_bam=None):
    found_in_both_parents=0
    if trios[patient] != {}:
        print(trios[patient])
    else:
        print(f'{patient} does not have relatives')
    inheritance=[]
    for rel in trios[patient].keys():
        if rel == 'child' or rel == 'mother' or rel == 'father':
            rel_num=int(trios[patient][rel])
            try:
                print(f'found {pos} in {rel_num} HC data - {gt_hc[rel_num][pos]}')
            except KeyError:
                print(f'{pos} not found in gt_hc of {rel_num} - the {rel} of {patient}')
                try:
                    print(f'found {pos} in {rel_num} DV data - {gt_dv[rel_num][pos]}')
                except KeyError:
                    print(f'{pos} not found in gt_dv of {rel_num} - the {rel} of {patient}')

            if pos in gt_hc[rel_num]:
                print(f'{pos} found in HC data of {rel}')
                hc_genotype=gt_hc[rel_num][pos][0]
                hc=hc_genotype.split('|')
                hc=int(hc[0])+int(hc[1])
                if hc == 2 and gt_hc[rel_num][pos][1]>=0.85:
                    if gt_hc[rel_num][pos][2]>=30:
                        print(f'found a homozygote {pos} in {rel} with VAF {gt_hc[rel_num][pos][1]}')
                        inheritance.append(hc_genotype)
                        if rel == 'mother' or rel == 'father':
                            found_in_both_parents+=1
                    else:
                        print(f'found a lowqual homozygote {pos} in {rel} with right VAF {gt_hc[rel_num][pos][1]}, but low DP - {gt_hc[rel_num][pos][2]}')
                        inheritance.append(hc_genotype)
                        if rel == 'mother' or rel == 'father':
                            found_in_both_parents+=1
                if hc == 1 and (gt_hc[rel_num][pos][1]>=0.2 and gt_hc[rel_num][pos][1]<0.85):
                    if gt_hc[rel_num][pos][2]>=30:
                        print(f'found a heterozygote {pos} in {rel} with VAF {gt_hc[rel_num][pos][1]}')
                        inheritance.append(hc_genotype)
                        if rel == 'mother' or rel == 'father':
                            found_in_both_parents+=1
                    else:
                        print(f'found a lowqual heterozygote {pos} in {rel} with right VAF {gt_hc[rel_num][pos][1]}, but low DP - {gt_hc[rel_num][pos][2]}')
                        inheritance.append(hc_genotype)
                        if rel == 'mother' or rel == 'father':
                            found_in_both_parents+=1
                else:
                    print(f'found only low qual {pos} in {rel} with {gt_hc[rel_num][pos][1]} and {gt_hc[rel_num][pos][2]}, probably an artefact')
            if pos in gt_dv[rel_num] and pos not in gt_hc[rel_num]:
                print(f'{pos} found only in DV data of {rel}')
                dv_genotype=gt_dv[rel_num][pos][0]
                dv=dv_genotype.split('/')
                dv=int(dv[0])+int(dv[1])
                if pos in gt_bam[rel_num].keys():
                    pp_value=gt_bam[rel_num][pos][0]
                    pp=pp_value.split('/')
                    pp=int(pp[0])+int(pp[1])
                    if pp==2 and gt_bam[rel_num][pos][1]>=0.85:
                        print(f'found a homozygote DV variant in {rel_num}')
                        inheritance.append(pp_value)
                        if rel == 'mother' or rel == 'father':
                            found_in_both_parents+=1
                    if pp==1 and (gt_bam[rel_num][pos][1]>=0.2 and gt_bam[rel_num][pos][1]<0.85):
                        print(f'found a heterozygote DV variant in {rel_num}')
                        inheritance.append(pp_value)
                        if rel == 'mother' or rel == 'father':
                            found_in_both_parents+=1
                # if dv == 2 and gt_dv[rel_num][pos][1]>=0.85:
                #     if gt_dv[rel_num][pos][2]>=30:
                #         print(f'found a homozygote {pos} in {rel} with VAF {gt_dv[rel_num][pos][1]}')
                #         inheritance.append(dv_genotype)
                #     else:
                #         print(f'found a lowqual homozygote {pos} in {rel} with right VAF {gt_dv[rel_num][pos][1]}, but low DP - {gt_dv[rel_num][pos][2]}')
                #         inheritance.append(dv_genotype)
                # if dv == 1 and (gt_dv[rel_num][pos][1]>=0.2 and gt_dv[rel_num][pos][1]<0.85):
                #     if gt_dv[rel_num][pos][2]>=30:
                #         print(f'found a heterozygote {pos} in {rel} with VAF {gt_dv[rel_num][pos][1]}')
                #         inheritance.append(dv_genotype)
                #     else:
                #         print(f'found a lowqual heterozygote {pos} in {rel} with right VAF {gt_dv[rel_num][pos][1]}, but low DP - {gt_dv[rel_num][pos][2]}')
                #         inheritance.append(dv_genotype)
                else:
                    print(f'cannot support found DV variant by bam, probably an artefact')

            if pos not in gt_hc[rel_num] and pos not in gt_dv[rel_num]:
                print(f'no {pos} found in {rel}')
    
    print(inheritance)
    if len(inheritance)>0:
        return True
    else:
        return False

In [11]:
def check_inheritance(patient, pos, trios, gt_hc, gt_dv):
    print(trios[patient])
    inheritance=[]
    for rel in trios[patient].keys():
        if rel == 'child' or rel == 'mother' or rel == 'father':
            rel_num=int(trios[patient][rel])
            try:
                print(f'found {pos} in {rel_num} HC data - {gt_hc[rel_num][pos]}')
            except KeyError:
                print(f'{pos} not found in gt_hc of {rel_num} - the {rel} of {patient}')
                try:
                    print(f'found {pos} in {rel_num} DV data - {gt_dv[rel_num][pos]}')
                except KeyError:
                    print(f'{pos} not found in gt_dv of {rel_num} - the {rel} of {patient}')
            if pos in gt_hc[rel_num] and pos in gt_dv[rel_num]:
                rel_gt_hc=gt_hc[rel_num][pos]
                rel_gt_dv = gt_dv[rel_num][pos]
                variant=check_relative_variant(pos, rel_gt_hc, rel_gt_dv)
                if variant!=None:
                    print(f'found {pos} in {rel} with genotype {variant}')
                    inheritance.append(variant)
            else:
                if pos in gt_hc[rel_num]:
                    print(f'{pos} found only in HC data of {rel}')
                    hc_genotype=gt_hc[rel_num][pos][0]
                    hc=hc_genotype.split('|')
                    hc=int(hc[0])+int(hc[1])
                    if hc == 2 and gt_hc[rel_num][pos][1]>=0.85:
                        if gt_hc[rel_num][pos][2]>=30:
                            print(f'found a homozygote {pos} in {rel} with VAF {gt_hc[rel_num][pos][1]}')
                            inheritance.append(hc_genotype)
                        else:
                            print(f'found a lowqual homozygote {pos} in {rel} with right VAF {gt_hc[rel_num][pos][1]}, but low DP - {gt_hc[rel_num][pos][2]}')
                            inheritance.append(hc_genotype)
                    if hc == 1 and (gt_hc[rel_num][pos][1]>=0.2 and gt_hc[rel_num][pos][1]<0.85):
                        if gt_hc[rel_num][pos][2]>=30:
                            print(f'found a heterozygote {pos} in {rel} with VAF {gt_hc[rel_num][pos][1]}')
                            inheritance.append(hc_genotype)
                        else:
                            print(f'found a lowqual heterozygote {pos} in {rel} with right VAF {gt_hc[rel_num][pos][1]}, but low DP - {gt_hc[rel_num][pos][2]}')
                            inheritance.append(hc_genotype)
                    else:
                        print(f'found only low qual {pos} in {rel} with {gt_hc[rel_num][pos][1]} and {gt_hc[rel_num][pos][2]}, probably an artefact')
                if pos in gt_dv[rel_num]:
                    print(f'{pos} found only in DV data of {rel}')
                    dv_genotype=gt_dv[rel_num][pos][0]
                    dv=dv_genotype.split('/')
                    dv=int(dv[0])+int(dv[1])
                    if dv == 2 and gt_dv[rel_num][pos][1]>=0.85:
                        if gt_dv[rel_num][pos][2]>=30:
                            print(f'found a homozygote {pos} in {rel} with VAF {gt_dv[rel_num][pos][1]}')
                            inheritance.append(dv_genotype)
                        else:
                            print(f'found a lowqual homozygote {pos} in {rel} with right VAF {gt_dv[rel_num][pos][1]}, but low DP - {gt_dv[rel_num][pos][2]}')
                            inheritance.append(dv_genotype)
                    if dv == 1 and (gt_dv[rel_num][pos][1]>=0.2 and gt_dv[rel_num][pos][1]<0.85):
                        if gt_dv[rel_num][pos][2]>=30:
                            print(f'found a heterozygote {pos} in {rel} with VAF {gt_dv[rel_num][pos][1]}')
                            inheritance.append(dv_genotype)
                        else:
                            print(f'found a lowqual heterozygote {pos} in {rel} with right VAF {gt_dv[rel_num][pos][1]}, but low DP - {gt_dv[rel_num][pos][2]}')
                            inheritance.append(dv_genotype)
                    else:
                        print(f'found only low qual {pos} in {rel} with {gt_dv[rel_num][pos][1]} and {gt_dv[rel_num][pos][2]}, probably an artefact')

                if pos not in gt_hc[rel_num] and pos not in gt_dv[rel_num]:
                    print(f'no {pos} found in {rel}')
    
    print(inheritance)
    if len(inheritance)>0:
        return True
    else:
        return False

In [12]:
#add DP checking
def check_variant(sample, pos, hc_genotype, dv_genotype, sample_hc_variants, sample_dv_variants, trios, gt_hc, gt_dv):
    hc=hc_genotype.split('|')
    hc=int(hc[0])+int(hc[1])
    dv=dv_genotype.split('/')
    dv=int(dv[0])+int(dv[1])
    if hc==dv:
        # print(f'{pos} are the same')
        return hc_genotype
    else:
        print(f'for {idx} comparing {pos} with HC - {hc_genotype} and DV - {dv_genotype}')
        if hc == 2:
            if sample_hc_variants[pos][1]>=0.85 and sample_hc_variants[pos][2]>=30:
                print(f'hc right with {hc_genotype}, VAF - {sample_hc_variants[pos][1]}, DP-{sample_hc_variants[pos][2]}')
                return hc_genotype
            if dv ==1 and (sample_dv_variants[pos][1] >= 0.2 and sample_dv_variants[pos][1] < 0.85) and sample_dv_variants[pos][2]>=30:
                print(f'dv right with {dv_genotype}, VAF - {sample_dv_variants[pos][1]}, DP-{sample_dv_variants[pos][2]}')
                return dv_genotype
            else:
                print(f'both wrong, checking inheritance; variant info: {hc_genotype}, {sample_hc_variants[pos][1]}, {sample_hc_variants[pos][2]} AND {dv_genotype}, {sample_dv_variants[pos][1]}, {sample_dv_variants[pos][2]}')
                inheritance = check_inheritance(sample, pos, hc_genotype, dv_genotype, sample_hc_variants, sample_dv_variants, trios, gt_hc, gt_dv)
                if inheritance == True:
                    print('saving this variant even with bad QUAL')
                    return hc_genotype
                else:
                    print('droping this variant, probably artefact')
        if hc == 1:
            if (sample_hc_variants[pos][1] >= 0.2 and sample_hc_variants[pos][1] < 0.85) and sample_hc_variants[pos][2]>=30:
                print(f'hc right with {hc_genotype}, VAF - {sample_hc_variants[pos][1]}, DP-{sample_hc_variants[pos][2]}')
                return hc_genotype
            if dv == 2 and sample_dv_variants[pos][1] >= 0.85 and sample_dv_variants[pos][2]>=30:
                print(f'dv right with {dv_genotype}, VAF - {sample_dv_variants[pos][1]}, DP-{sample_dv_variants[pos][2]}')
                return dv_genotype
            else:
                print(f'both wrong, checking inheritance; variant info: {hc_genotype}, {sample_hc_variants[pos][1]}, {sample_hc_variants[pos][2]} AND {dv_genotype}, {sample_dv_variants[pos][1]}, {sample_dv_variants[pos][2]}')
                inheritance = check_inheritance(sample, pos, trios, gt_hc, gt_dv)
                if inheritance == True:
                    print('saving this variant even with bad QUAL')
                    return hc_genotype
                else:
                    print('droping this variant, probably artefact')

In [10]:
gt_hc=defaultdict(dict)
gt_dv=defaultdict(dict)
gt_bam=defaultdict(dict)
for idx, row in files_list.iterrows():
    # dv
    vcf = read_vcf(row['dv'], mode = 'dv')
    vcf = vcf[vcf['FILTER'] == 'PASS']
    vcf = vcf[(vcf['POS'] >= calling_region[0]) & (vcf['POS'] <= calling_region[1])]
    gt_dv[idx]=gt_vectors(vcf, mode = 'dv')
    # hc 
    vcf = read_vcf(row['hc'], mode='hc')
    vcf = vcf[(vcf['POS'] >= calling_region[0]) & (vcf['POS'] <= calling_region[1])]
    gt_hc[idx]=gt_vectors(vcf, mode= 'hc')
    #bam
    vcf = read_vcf(row['pileup'], mode='bam')
    vcf = vcf[(vcf['POS'] >= calling_region[0]) & (vcf['POS'] <= calling_region[1])]
    gt_bam[idx]=gt_vectors(vcf, mode= 'dv')


In [11]:
res = defaultdict(dict)
for idx, row in files_list.iterrows():
    # dv
    vcf = read_vcf(row['dv'], mode = 'dv')
    vcf = vcf[vcf['FILTER'] == 'PASS']
    vcf = vcf[(vcf['POS'] >= calling_region[0]) & (vcf['POS'] <= calling_region[1])]
    sample_dv=gt_vectors(vcf, mode = 'dv')
    # hc 
    vcf = read_vcf(row['hc'], mode='hc')
    vcf = vcf[(vcf['POS'] >= calling_region[0]) & (vcf['POS'] <= calling_region[1])]
    sample_hc=gt_vectors(vcf, mode= 'hc')

    #pileup
    vcf = read_vcf(row['pileup'], mode='bam')
    vcf = vcf[(vcf['POS'] >= calling_region[0]) & (vcf['POS'] <= calling_region[1])]
    sample_pp=gt_vectors(vcf, mode= 'dv')

    print(f'checking variants for {idx}')
    for position in sample_hc.keys():
        hc_value=sample_hc[position][0]
        hc=hc_value.split('|')
        hc=int(hc[0])+int(hc[1])
        if position in sample_dv.keys():
            dv_value=sample_dv[position][0]
            dv=dv_value.split('/')
            dv=int(dv[0])+int(dv[1])
            if hc == dv:
                # print(f'no conflict for {position} in {idx}, saving HC genotype')
                res[idx][position] = hc_value
            if hc!=dv:
                if hc == 2:
                    if sample_hc[position][1] > 0.85 and sample_hc[position][2]>=30:
                        res[idx][position] = hc_value
                    else:
                        print(f'check {position}: HC - {sample_hc[position]}, DV - {dv_value}, bam - {sample_pp.get(position, 'Not found in bam')}')
                        if position in sample_pp.keys():
                            pp_value=sample_pp[position][0]
                            pp=pp_value.split('/')
                            pp=int(pp[0])+int(pp[1])
                            if pp==2 and sample_pp[position][1]>=0.85:
                                res[idx][position]  = hc_value
                                print(f'HC genotype is supported by bam')
                            if pp==1 and (sample_pp[position][1]>=0.2 and sample_pp[position][1]<0.85):
                                print(f'HC genotype is not supported by bam, saving a new genotype')
                                print(sample_pp[position])
                                res[idx][position] = pp_value
                        else:
                            print('need to check inheritance')
                            inheritance = check_inheritance(idx, position, trios, gt_hc, gt_dv, gt_bam)
                            if inheritance == True:
                                print('saving this variant even with bad QUAL')
                                res[idx][position]  = hc_value
                            else:
                                print('droping this variant, probably artefact')
                                
                if hc == 1: 
                    if (sample_hc[position][1] >= 0.2 and sample_hc[position][1] < 0.85) and sample_hc[position][2]>=30:
                        res[idx][position]  = hc_value
                    else:
                        print(f'check {position}: HC - {sample_hc[position]}, DV - {dv_value}, {sample_pp.get(position, 'Not found in bam')}')
                        if position in sample_pp.keys():
                            pp_value=sample_pp[position][0]
                            pp=pp_value.split('/')
                            pp=int(pp[0])+int(pp[1])
                            if pp==1 and (sample_pp[position][1]>=0.2 and sample_pp[position][1]<0.85):
                                res[idx][position]  = hc_value
                                print(f'HC genotype is supported by bam')
                            if pp==2 and sample_pp[position][1]>=0.85:
                                print(f'HC genotype is not supported by bam, saving a new genotype')
                                print(sample_pp[position])
                                res[idx][position] = pp_value
                        else:
                            print('need to check inheritance')
                            inheritance = check_inheritance(idx, position, trios, gt_hc, gt_dv, gt_bam)
                            if inheritance == True:
                                print('saving this variant even with bad QUAL')
                                res[idx][position]  = hc_value
                            else:
                                print('droping this variant, probably artefact')
        else:
            hc_value=sample_hc[position][0]
            hc=hc_value.split('|')
            hc=int(hc[0])+int(hc[1])
            if hc == 2:
                if sample_hc[position][1] >= 0.85 and sample_hc[position][2]>=30:
                    res[idx][position]  = hc_value
                else:
                    print(idx)
                    print(f'check {position}: HC - {sample_hc[position]}, DV - not called, bam - {sample_pp.get(position, 'Not found in bam')}')
                    if position in sample_pp.keys():
                        pp_value=sample_pp[position][0]
                        pp=pp_value.split('/')
                        pp=int(pp[0])+int(pp[1])
                        if pp==2 and sample_pp[position][1]>=0.85:
                            res[idx][position]  = hc_value
                            print(f'genotype is supported by bam')
                        if pp==1 and (sample_pp[position][1]>=0.2 and sample_pp[position][1]<0.85):
                            print(f'HC genotype is not supported by bam, saving a new genotype')
                            res[idx][position]  = pp_value
                    else:
                        print('need to check inheritance')
                        inheritance = check_inheritance(idx, position, trios, gt_hc, gt_dv, gt_bam)
                        if inheritance == True:
                            print('saving this variant even with bad QUAL')
                            res[idx][position]  = hc_value
                        else:
                            print('droping this variant, probably artefact')
            if hc == 1:
                if (sample_hc[position][1] >= 0.2 and sample_hc[position][1] < 0.85):
                    res[idx][position]  = hc_value
                else:             
                    print(idx)
                    print(f'check {position}: HC - {sample_hc[position]}, DV - not called, {sample_pp.get(position, 'Not found in bam')}')
                    if position in sample_pp.keys():
                        pp_value=sample_pp[position][0]
                        pp=pp_value.split('/')
                        pp=int(pp[0])+int(pp[1])
                        if hc == pp and (sample_pp[position][1] >= 0.2 and sample_pp[position][1]<0.85):
                            res[idx][position]  = hc_value
                            print(f'HC genotype is supported by bam')
                        else:
                            print(f'HC genotype is not supported by bam, saving a new genotype')
                            print(sample_pp[position])
                            res[idx][position]  = pp_value
                    else:
                        print('need to check inheritance')
                        inheritance = check_inheritance(idx, position, trios, gt_hc, gt_dv, gt_bam)
                        if inheritance == True:
                            print('saving this variant even with bad QUAL')
                            res[idx][position]  = hc_value
                        else:
                            print('droping this variant, probably artefact')
    for position in sample_dv.keys():
        if position not in res[idx].keys():
            print(f'{position} in {idx} found in DV, but not in HC, checking it')
            dv_value=sample_dv[position][0]
            dv=dv_value.split('/')
            dv=int(dv[0])+int(dv[1])
            if dv == 2:
                if position in sample_pp.keys():
                    if sample_pp[position][1]> 0.85:
                        print(f'{position} not called by HC, saving it with {dv_value}, {sample_pp[position]}')
                        res[idx][position] = dv_value
                else:
                    print(f'low qual {position} in {idx}, checking inheritance; variant info: {dv_value}')
                    inheritance = check_inheritance(idx, position, trios, gt_hc, gt_dv, gt_bam)
                    if inheritance == True:
                        print('saving this variant even with bad QUAL')
                        res[idx][position] = dv_value
                    else:
                        print('droping this variant, probably artefact')
            if dv == 1:
                if position in sample_pp.keys():
                    if (sample_pp[position][1] >= 0.2 and sample_pp[position][1] < 0.85):
                        print(f'{position} not called by HC, saving it with {dv_value}, {sample_pp[position]}')
                        res[idx][position] = dv_value
                else:
                    print(f'low qual {position} in {idx}, checking inheritance; variant info: {dv_value}')
                    inheritance = check_inheritance(idx, position, trios, gt_hc, gt_dv, gt_bam)
                    if inheritance == True:
                        print('saving this variant even with bad QUAL')
                        res[idx][position] = dv_value
                    else:
                        print('droping this variant, probably artefact')

checking variants for 77050386
checking variants for 77091048
checking variants for 77092306
checking variants for 77092307
checking variants for 77092308
checking variants for 77092309
checking variants for 77092310
checking variants for 77092311
checking variants for 77092312
32039081_C_G in 77092312 found in DV, but not in HC, checking it
32039081_C_G not called by HC, saving it with 1/1, ['1/1', 0.997539975399754, 1626]
checking variants for 77092313
checking variants for 77092314
checking variants for 77092315
checking variants for 77092316
checking variants for 77092317
checking variants for 77092319
checking variants for 77092320
checking variants for 77092321
checking variants for 77092322
checking variants for 77092323
checking variants for 77092324
checking variants for 77092327
checking variants for 77092329
checking variants for 77092344
checking variants for 77092345
checking variants for 77092346
checking variants for 77092347
checking variants for 77092348
checking varia

In [12]:
trios[77092370]

{'father': 77092368, 'mother': 77092371}

In [15]:
print(gt_hc[77092370]['32038949_T_G'], 'NONE', gt_dv[77092370]['32038949_T_G'], gt_dv[77092370]['32038949_T_C'])

KeyboardInterrupt: 

In [67]:
gt_bam[77092898]

{'32038115_G_C': ['1/1', 0.9778085991678225, 1442],
 '32038127_T_C': ['1/1', 0.9961180124223602, 1288],
 '32038128_A_C': ['1/1', 0.9992223950233281, 1286],
 '32038139_A_G': ['1/1', 0.996177370030581, 1308],
 '32038141_T_G': ['1/1', 1.0, 1365],
 '32038213_T_C': ['1/1', 0.9944328462073765, 1437],
 '32038224_C_T': ['1/1', 0.996395097332372, 1387],
 '32038297_C_T': ['1/1', 0.9927826784282278, 1247],
 '32038310_G_A': ['1/1', 0.9694589877835951, 1146],
 '32038313_T_C': ['1/1', 1.0, 1117],
 '32038320_A_G': ['1/1', 0.9873843566021867, 1189],
 '32038419_C_T': ['1/1', 0.9975339087546239, 811],
 '32038514_C_T': ['1/1', 0.9549653579676675, 866],
 '32038844_A_C': ['1/1', 0.8655083655083655, 1554],
 '32038857_A_T': ['1/1', 0.9485294117647058, 544],
 '32038867_T_G': ['1/1', 0.9952076677316294, 1252],
 '32038878_C_T': ['1/1', 0.9951923076923077, 1248],
 '32038895_A_G': ['1/1', 0.9974619289340102, 1182],
 '32038903_A_G': ['1/1', 0.9982062780269059, 1115],
 '32038911_A_G': ['1/1', 0.9943872778297475, 10

In [ ]:
res = defaultdict(dict)
for idx, row in files_list.iterrows():
    # dv
    vcf = read_vcf(row['dv'], mode = 'dv')
    vcf = vcf[vcf['FILTER'] == 'PASS']
    vcf = vcf[(vcf['POS'] >= calling_region[0]) & (vcf['POS'] <= calling_region[1])]
    sample_dv=gt_vectors(vcf, mode = 'dv')
    # hc 
    vcf = read_vcf(row['hc'], mode='hc')
    vcf = vcf[(vcf['POS'] >= calling_region[0]) & (vcf['POS'] <= calling_region[1])]
    sample_hc=gt_vectors(vcf, mode= 'hc')

    #pileup
    vcf = read_vcf(row['pileup'], mode='bam')
    vcf = vcf[(vcf['POS'] >= calling_region[0]) & (vcf['POS'] <= calling_region[1])]
    sample_pp=gt_vectors(vcf, mode= 'dv')

    for position in sample_hc.keys():
        hc_value=sample_hc[position][0]
        if position in sample_dv.keys():
            dv_value=sample_dv[position][0]
            res[idx][position] = check_variant(idx, position, hc_value, dv_value, sample_hc, sample_dv, trios, gt_hc, gt_dv)
        else:
            hc_value=sample_hc[position][0]
            hc=hc_value.split('|')
            hc=int(hc[0])+int(hc[1])
            if hc == 2:
                if sample_hc[position][1]>0.85 and sample_hc[position][2]>=30:
                    # print(f'{position} not called by DV, saving it with {hc_value}, {sample_hc[position][1]}, {sample_hc[position][2]}')
                    res[idx][position] = hc_value
                else:
                    print(f'low qual {position} in {idx}, checking inheritance; variant info: {hc_value}, {sample_hc[position][1]}, {sample_hc[position][2]}')
                    inheritance = check_inheritance(idx, position, trios, gt_hc, gt_dv)
                    if inheritance == True:
                        print('saving this variant even with bad QUAL')
                        res[idx][position] = hc_value
                    else:
                        print('droping this variant, probably artefact')
            if hc == 1:
                if (sample_hc[position][1] >= 0.2 and sample_hc[position][1] < 0.85) and sample_hc[position][2]>=30:
                    # print(f'{position} not called by DV, saving it with {hc_value}, {sample_hc[position][1]}, {sample_hc[position][2]}')
                    res[idx][position] = hc_value
                else:
                    print(f'low qual {position} in {idx}, checking inheritance; variant info: {hc_value}, {sample_hc[position][1]}, {sample_hc[position][2]}')
                    inheritance = check_inheritance(idx, position, trios, gt_hc, gt_dv)
                    if inheritance == True:
                        print('saving this variant even with bad QUAL')
                        res[idx][position] = hc_value
                    else:
                        print('droping this variant, probably artefact')
    for position in sample_dv.keys():
        if position not in res[idx].keys():
            dv_value=sample_dv[position][0]
            dv=dv_value.split('/')
            dv=int(dv[0])+int(dv[1])
            if dv == 2:
                if sample_dv[position][1]> 0.85 and sample_dv[position][2]>=30:
                    # print(f'{position} not called by HC, saving it with {dv_value}, {sample_dv[position][1]}, {sample_dv[position][2]}')
                    res[idx][position] = dv_value
                else:
                    print(f'low qual {position} in {idx}, checking inheritance; variant info: {dv_value}, {sample_dv[position][1]}, {sample_dv[position][2]}')
                    inheritance = check_inheritance(idx, position, trios, gt_hc, gt_dv)
                    if inheritance == True:
                        print('saving this variant even with bad QUAL')
                        res[idx][position] = dv_value
                    else:
                        print('droping this variant, probably artefact')
            if dv == 1:
                if (sample_dv[position][1] >= 0.2 and sample_dv[position][1] < 0.85) and sample_dv[position][2]>=30:
                    # print(f'{position} not called by HC, saving it with {dv_value}, {sample_dv[position][1]}, {sample_dv[position][2]}')
                    res[idx][position] = dv_value
                else:
                    print(f'low qual {position} in {idx}, checking inheritance; variant info: {dv_value}, {sample_dv[position][1]}, {sample_dv[position][2]}')
                    inheritance = check_inheritance(idx, position, trios, gt_hc, gt_dv)
                    if inheritance == True:
                        print('saving this variant even with bad QUAL')
                        res[idx][position] = dv_value
                    else:
                        print('droping this variant, probably artefact')

for 77092312 comparing 32038855_T_TTTG with HC - 1|0 and DV - 1/1
hc right with 1|0, VAF - 0.8401396160558464, DP-2868
low qual 32038855_T_TTG in 77092312, checking inheritance; variant info: 1|0, 0.15986038394415358, 2868
{'mother': 77092311, 'sibling': 77092313}
32038855_T_TTG not found in gt_hc of 77092311 - the mother of 77092312
32038855_T_TTG not found in gt_dv of 77092311 - the mother of 77092312
no 32038855_T_TTG found in mother
[]
droping this variant, probably artefact
low qual 32038855_T_TTG in 77092329, checking inheritance; variant info: 1|0, 0.18454661558109833, 1581
{'mother': 77092327}
32038855_T_TTG not found in gt_hc of 77092327 - the mother of 77092329
32038855_T_TTG not found in gt_dv of 77092327 - the mother of 77092329
no 32038855_T_TTG found in mother
[]
droping this variant, probably artefact
for 77092329 comparing 32038855_T_TTTG with HC - 1|0 and DV - 1/1
hc right with 1|0, VAF - 0.8154533844189017, DP-1581
for 77092370 comparing 32038855_T_TTTG with HC - 1|0 

In [17]:
left_aligned=defaultdict(dict)
for idx, row in files_list.iterrows():
    # dv
    vcf = read_vcf(row['dv'], mode = 'dv')
    vcf = vcf[vcf['FILTER'] == 'PASS']
    vcf = vcf[(vcf['POS'] >= calling_region[0]) & (vcf['POS'] <= calling_region[1])]
    if 32038855 in list(vcf['POS']):
        left_aligned[str(idx)+'_DV']={'POS': 32038855, 
                           'REF': vcf.loc[vcf['POS']==32038855, 'REF'].values[0],
                           'ALT': vcf.loc[vcf['POS']==32038855, 'ALT'].values[0],
                           'GT': vcf.loc[vcf['POS']==32038855, 'GT'].values[0],
                           'AD': vcf.loc[vcf['POS']==32038855, 'AD'].values[0]}
    # hc 
    vcf = read_vcf(row['hc'], mode='hc')
    vcf = vcf[(vcf['POS'] >= calling_region[0]) & (vcf['POS'] <= calling_region[1])]
    if 32038855 in list(vcf['POS']):
        left_aligned[str(idx)+'_HC']={'POS': 32038855, 
                           'REF': vcf.loc[vcf['POS']==32038855, 'REF'].values[0],
                           'ALT': vcf.loc[vcf['POS']==32038855, 'ALT'].values[0],
                           'GT': vcf.loc[vcf['POS']==32038855, 'GT'].values[0],
                           'AD': vcf.loc[vcf['POS']==32038855, 'AD'].values[0]}
    # gt_hc[idx]=gt_vectors(vcf, mode= 'hc')

In [18]:
conf=pd.DataFrame.from_dict(conflicts).transpose()
conf.to_excel('/storage/haplotypes/excels/conflicts.xlsx')

In [47]:
df=pd.DataFrame.from_dict(res)
df.columns = df.columns.astype(str)
df['pos'] = df.index.to_series().str.extract(r'^(\d+)').astype(int)
df['rest'] = df.index.to_series().str.extract(r'^\d+_(.+)$')
df_sorted = df.sort_values(by=['pos', 'rest']).drop(columns=['pos', 'rest'])
df_sorted.to_excel('/storage/haplotypes/excels/not_phased_sorted.xlsx')

In [16]:
df_af=df.copy()
df_af.head()

,77050386,77091048,77092306,77092307,77092308,77092309,77092310,77092311,77092312,77092313,...,77095381,77095382,77095383,77095384,77095385,77095386,77095387,77095388,77095389,77095390
32038302_C_T,0|1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32038437_CCTG_C,1|0,NaN,NaN,NaN,NaN,NaN,NaN,0|1,NaN,1|0,...,0|1,NaN,NaN,NaN,NaN,1|1,1|1,1|0,NaN,NaN
32038540_C_T,1|1,1|1,1|0,0|1,1|1,NaN,0|1,0|1,NaN,1|0,...,0|1,0|1,1|0,NaN,1|1,1|1,1|1,1|0,0|1,1|0
32038560_C_A,1|1,0|1,1|0,0|1,1|1,NaN,0|1,0|1,NaN,1|0,...,0|1,0|1,1|0,NaN,1|1,1|1,1|1,1|0,NaN,1|0
32038820_C_T,0|1,1|1,NaN,0|1,0|1,NaN,1|0,0|1,NaN,1|0,...,1|0,NaN,NaN,NaN,NaN,1|1,1|1,0|1,1|1,1|0


In [82]:
to_phase= df.loc[:, df.apply(lambda col: col.isin(['1/0', '0/1']).any())]
to_phase.to_excel('/storage/haplotypes/excels/to_phase.xlsx')

In [16]:
def process_cell(value):
    if pd.isna(value):
        return 0
    # Replace both '/' and '|' with '+' and evaluate
    try:
        return eval(str(value).replace('/', '+').replace('|', '+'))
    except:
        return 0 

In [17]:
processed_df = df.applymap(process_cell)
processed_df['haplotype_count'] = processed_df.sum(axis=1)
processed_df['AF_cohort'] = processed_df['haplotype_count']/1142
processed_df.to_excel('/storage/haplotypes/excels/w_af.xlsx')



/tmp/ipykernel_2772568/3509597075.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  processed_df = df.applymap(process_cell)


In [18]:
trios[77092370]

{'father': 77092368, 'mother': 77092371}

In [21]:
print(gt_hc[77092370]['32038949_T_G'], 'NONE', gt_dv[77092370]['32038949_T_G'], gt_dv[77092370]['32038949_T_C'], gt_bam[77092370]['32038949_T_G'], gt_bam[77092370]['32038949_T_C'])

['1|1', 1.0, 25] NONE ['1/0', 0.7857142857142857, 168] ['1/0', 0.19642857142857142, 168] ['1/0', 0.35555555555555557, 90] ['1/0', 0.5666666666666667, 90]


In [36]:
print(gt_hc[77092368]['32038949_T_G'], 'NONE', 'NONE', 'NONE', 'NONE', 'NONE')

['1|0', 0.5511111111111111, 225] NONE NONE NONE NONE NONE


In [37]:
print(gt_hc[77092371]['32038949_T_G'], 'NONE', 'NONE', 'NONE', gt_bam[77092371]['32038949_T_G'], 'NONE')

['1|0', 0.5049035115469788, 3161] NONE NONE NONE ['0/1', 0.26745240253853125, 1103] NONE


In [ ]:
trios[77095354] 

{'mother': 77095347, 'sibling': 77095349, 'father': 77095350}

In [56]:
print('HC 32039015_G_T NONE', gt_hc[77095354]['32039015_G_A'], 'DV 32039015_G_T NONE', gt_dv[77095354]['32039015_G_A'], gt_bam[77095354]['32039015_G_T'], gt_bam[77095354]['32039015_G_A'])

HC 32039015_G_T NONE ['1|1', 1.0, 46] DV 32039015_G_T NONE ['0/1', 0.7916666666666666, 72] ['1/0', 0.3144208037825059, 846] ['1/0', 0.5638297872340425, 846]


In [ ]:
print(gt_hc[77095354]['32039015_G_T'], gt_hc[77095354]['32039015_G_A'], gt_hc[77095354]['32039015_G_T'], gt_dv[77095354]['32039015_G_A'], gt_bam[77095354]['32039015_G_T'], gt_bam[77095354]['32039015_G_A'])

In [63]:
def manual_checking(patient, pos1, pos2, relatives):
    print(gt_hc[patient].get(pos1, f'{pos1} not found in HC data of {patient}'))
    print(gt_dv[patient].get(pos1, f'{pos1} not found in DV data of {patient}'))
    print(gt_bam[patient].get(pos1, f'{pos1} not found in bam data of {patient}'))
    print(gt_hc[patient].get(pos2, f'{pos2} not found in HC data of {patient}'))
    print(gt_dv[patient].get(pos2, f'{pos1} not found in DV data of {patient}'))
    print(gt_bam[patient].get(pos2, f'{pos1} not found in bam data of {patient}'))
    for rel in relatives[patient]:
        rel_num=int(relatives[patient][rel])
        print(f'searching in {rel_num}  - the {rel} of {patient}')
        print(gt_hc[rel_num].get(pos1, f'{pos1} not found in HC data of {rel_num}'))
        print(gt_dv[rel_num].get(pos1, f'{pos1} not found in DV data of {rel_num}'))
        print(gt_bam[rel_num].get(pos1, f'{pos1} not found in bam data of {rel_num}'))
        print(gt_hc[rel_num].get(pos2, f'{pos2} not found in HC data of {rel_num}'))
        print(gt_dv[rel_num].get(pos2, f'{pos1} not found in DV data of {rel_num}'))
        print(gt_bam[rel_num].get(pos2, f'{pos1} not found in bam data of {rel_num}'))


    return None

In [65]:
manual_checking(77092531, '32039002_G_A', '32039015_G_A', trios)

32039002_G_A not found in HC data of 77092531
['1/1', 0.9487179487179487, 39]
32039002_G_A not found in bam data of 77092531
32039015_G_A not found in HC data of 77092531
32039002_G_A not found in DV data of 77092531
32039002_G_A not found in bam data of 77092531
searching in 77092530  - the father of 77092531
['0|1', 0.23255813953488372, 43]
32039002_G_A not found in DV data of 77092530
32039002_G_A not found in bam data of 77092530
32039015_G_A not found in HC data of 77092530
32039002_G_A not found in DV data of 77092530
32039002_G_A not found in bam data of 77092530
searching in 77092529  - the mother of 77092531
['0|1', 0.125, 24]
32039002_G_A not found in DV data of 77092529
32039002_G_A not found in bam data of 77092529
32039015_G_A not found in HC data of 77092529
32039002_G_A not found in DV data of 77092529
32039002_G_A not found in bam data of 77092529
